    # Data Processing

In [ ]:
import pandas as pd
import datetime
import time
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns  #Plots
import pickle #to save the model
import warnings

In [ ]:
%%bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv
wget https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc\?method\=export\&format\=GeoJSON -O taxi_zones.geojson

In [ ]:
%%bash
for i in {1..12}
do
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-$(printf "%02d" $i).parquet
done
mkdir yellow_tripdata_2022
mv yellow_tripdata_2022-*.parquet yellow_tripdata_2022


In [ ]:

import datetime
import pandas as pd

trips = pd.read_parquet(
    'yellow_tripdata_2022',
    columns=[
        'tpep_pickup_datetime',
        'tpep_dropoff_datetime',
        'trip_distance',
        'PULocationID',
        'DOLocationID',
    ]
    )
# Remove outliers.
trips = trips[(trips['tpep_pickup_datetime'] >= datetime.datetime(2022, 1, 1)) & (trips['tpep_pickup_datetime'] <= datetime.datetime(2022, 12, 31))]
trips.head()


In [ ]:
trips.dtypes

In [ ]:
trips.shape

In [ ]:
taxi_zones = pd.read_csv('taxi+_zone_lookup.csv', usecols=['LocationID', 'Zone'])
taxi_zones.set_index(['LocationID'], inplace=True)
taxi_zones

In [ ]:
import json
with open('taxi_zones.geojson') as fd:
    geojson = json.load(fd)

## Temporal Analysis




In [ ]:
trips['PU_dayofweek'] = trips['tpep_pickup_datetime'].dt.dayofweek
trips['PU_hour'] = trips['tpep_pickup_datetime'].dt.hour
gb_time = trips.groupby(by=['PU_dayofweek', 'PU_hour'], as_index=False).agg(count=('PU_dayofweek', 'count'))
gb_time

In [ ]:
import plotly.express as px

b = px.bar(
    gb_time,
    x='PU_hour',
    y='count',
    color='PU_dayofweek',
    color_continuous_scale='sunset_r',
)
b.show()

In [ ]:
b = px.bar(
    gb_time,
    x='PU_dayofweek',
    y='count',
    color='PU_hour',
    color_continuous_scale='sunset_r',
)
b.show()

Spatial Analysis


## Spatial Analysis


In [ ]:
gb_pu_location = trips.groupby(['PULocationID'], as_index=False).agg(count=('PULocationID', 'count'))
gb_pu_location

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    go.Choroplethmapbox(
        geojson=geojson,
        featureidkey='properties.location_id',
        locations=gb_pu_location['PULocationID'],
        z=gb_pu_location['count'],
        colorscale="Viridis",
        marker_opacity=0.7,
        marker_line_width=0.1
    )
)
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=9,
    mapbox_center = {"lat": 40.7158, "lon": -73.9805},
    height=600,
)
fig.show()

In [2]:
%%bash
for i in {1..12}
do
    wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-$(printf "%02d" $i).parquet
done
mkdir yellow_tripdata_2016
mv yellow_tripdata_2016-*.parquet yellow_tripdata_2016

--2024-04-23 19:45:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2016-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 108.156.171.190, 108.156.171.231, 108.156.171.75, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|108.156.171.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151251087 (144M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2016-01.parquet’

     0K .......... .......... .......... .......... ..........  0% 1.99M 73s
    50K .......... .......... .......... .......... ..........  0% 2.38M 67s
   100K .......... .......... .......... .......... ..........  0% 4.06M 56s
   150K .......... .......... .......... .......... ..........  0% 9.23M 46s
   200K .......... .......... .......... .......... ..........  0% 4.47M 43s
   250K .......... .......... .......... .......... ..........  0% 12.5M 38s
   300K .......... .......... .........

In [1]:
import datetime
import pandas as pd

trips = pd.read_parquet(
    'yellow_tripdata_2016'
    )
# Remove outliers.
trips = trips[(trips['tpep_pickup_datetime'] == datetime.datetime(2016, 1, 1))]
trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
8167,2,2016-01-01,2016-01-01 00:00:00,1,10.54,1,N,106,75,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3,None,None
11997,2,2016-01-01,2016-01-01 00:00:00,5,4.90,1,N,79,80,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3,None,None
12978,2,2016-01-01,2016-01-01 00:00:00,3,1.76,1,N,43,161,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8,None,None
14641,2,2016-01-01,2016-01-01 00:26:45,2,7.45,1,N,148,238,2,26.0,0.5,0.5,0.0,0.0,0.3,27.3,None,None
17048,2,2016-01-01,2016-01-01 00:18:30,2,5.52,1,N,170,7,2,19.0,0.5,0.5,0.0,0.0,0.3,20.3,None,None


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


X = trips[['PU_hour', 'PULocationID']]  
y = trips['trip_duration']  


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 创建一个线性回归模型
model = LinearRegression()

# 训练模型
model.fit(X_train, y_train)

# 预测测试集结果
y_pred = model.predict(X_test)

# 评估模型
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 使用模型进行预测
# 例如预测 PU_hour = 8, PULocationID = 132 的情况
sample_data = pd.DataFrame({'PU_hour': [8], 'PULocationID': [132]})
predicted_duration = model.predict(sample_data)
print(f"Predicted trip duration: {predicted_duration[0]} minutes")


NameError: name 'trips' is not defined